In [2]:
import json
import os
from pathlib import Path
from typing import Annotated, Optional
from uuid import uuid4

import anthropic
import instructor
import openai
from dotenv import load_dotenv
from pydantic import BaseModel, BeforeValidator, Field, field_validator, model_validator

from dreamai.ai import ModelName, assistant_message, system_message, user_message
from dreamai.utils import deindent

load_dotenv()

ask_oai = instructor.from_openai(openai.OpenAI())
ask_cld = instructor.from_anthropic(anthropic.Anthropic())

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
class Info(BaseModel):
    answer: str = Field(
        ..., description="The answer to the user's question about the podcast."
    )
    timestamps: list[float] = Field(
        ..., description="The timestamps in the transcript where the answer was found."
    )

    @field_validator("timestamps")
    @classmethod
    def sort_timestamps(cls, timestamps) -> list[float]:
        return sorted(timestamps)

In [51]:
def merge_same_role_messages(messages: list[dict]) -> list[dict]:
    if not messages:
        return []
    new_messages = []
    last_message = None
    for message in messages:
        if last_message is None:
            last_message = message
        elif last_message["role"] == message["role"]:
            last_message["content"] += "\n\n" + message["content"]
        else:
            new_messages.append(last_message)
            last_message = message
    if last_message is not None:
        new_messages.append(last_message)
    return new_messages


def get_segments_between(
    segments: list[dict], start: float = 0, end: float = float("inf")
) -> list[dict]:
    return [
        segment
        for segment in segments
        if segment["start"] >= start and segment["start"] <= end
    ]


def merge_consecutive_segments(segments: list[dict]) -> list[dict]:
    merged_segments = []
    for segment in segments:
        if not merged_segments:
            merged_segments.append(segment)
        else:
            last_segment = merged_segments[-1]
            if last_segment["end"] == segment["start"]:
                last_segment["end"] = segment["end"]
            else:
                merged_segments.append(segment)
    return merged_segments

In [33]:
sys_message = deindent(
    """
You are an avid listener of podcasts and you have world class information retention.
You will be a given a part of a transcript from a podcast episode and you will be asked to answer questions based on the information in the transcript.
Also return the timestamp of the segment in the podcast where the answer was found.
"""
)

In [55]:
segments = json.load(open("segments.json"))

In [34]:
timestamp = 60
segments_until_timestamp = get_segments_between(segments, end=timestamp)

In [40]:
transcript_message = user_message(
    f"TRANSCRIPT UNTIL {timestamp} SECONDS\n\n{json.dumps(segments_until_timestamp)}"
)
print(transcript_message["content"])

TRANSCRIPT UNTIL 60 SECONDS

[{"start": 0.0, "end": 2.72, "text": " The Athletic"}, {"start": 2.72, "end": 14.36, "text": " Hello, welcome back to the athletic football tactics podcast,"}, {"start": 14.82, "end": 19.0, "text": " where this week we are talking about superstars and systems"}, {"start": 19.0, "end": 23.06, "text": " in an era where superstar status is still rampant in media coverage"}, {"start": 23.06, "end": 25.080000000000002, "text": " and the fan experience around football,"}, {"start": 25.42, "end": 28.98, "text": " is that reflected with what we see on the pitch at the top level?"}, {"start": 28.98, "end": 31.7, "text": " No superstars here in the studio,"}, {"start": 31.86, "end": 35.26, "text": " just a trio of athletic writers with perfect alchemy"}, {"start": 35.26, "end": 38.56, "text": " Mark Carey, Liam Thumb and Michael Cox, join me."}, {"start": 38.64, "end": 39.06, "text": " Hi, guys."}, {"start": 39.18, "end": 40.18, "text": " Hi, Ali."}, {"start": 40.84,

In [41]:
query_message = user_message("Who is on today's episode?")

messages = [transcript_message, query_message]

In [46]:
info = ask_cld.create(
    system=sys_message,
    messages=merge_same_role_messages(messages),  # type: ignore
    model=ModelName.HAIKU,
    max_tokens=1024,
    response_model=Info,
)

In [47]:
info

Info(answer="The people on today's episode are Mark Carey, Liam Thumb, and Michael Cox.", timestamps=[35.26, 38.56])

In [53]:
info_segments = get_segments_between(
    segments=segments, start=info.timestamps[0], end=info.timestamps[-1]
)
info_segments

[{'start': 35.26,
  'end': 38.56,
  'text': ' Mark Carey, Liam Thumb and Michael Cox, join me.'}]